In [39]:
import pandas as pd   #used for data manipulation and analysis
import numpy as np    #used for mathematical and logical operations
import warnings
warnings.filterwarnings('ignore')

In [40]:
sample = pd.read_csv('sample.csv')
df=sample

## 1 
#std, scale, remove missing, i.t.n => same as best model
#preprocessed_sample = ......

In [41]:
#Delime na 2 mnozestva. prvoto>
df_num = df.select_dtypes(include = ['float64', 'int64'])

In [42]:
# tuka se kategoriski , vtoroto:
df_cat = df.select_dtypes(include = ['object'])

In [43]:
df_num['Months_on_book']=df_num.Months_on_book.replace({-2147483647.0:np.nan,
                                                       2147483647.0:np.nan,
                                                       3.21000000e+11:np.nan})

In [44]:
import sklearn
from sklearn.impute import SimpleImputer

In [45]:
mob_imputer = SimpleImputer(missing_values = np.NaN, strategy = 'mean') # go definirame Simple imputer, stratgegijata = mean
df_num.Months_on_book = mob_imputer.fit_transform(df_num['Months_on_book'].values.reshape(-1,1))[:,0]

In [46]:
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler # za varijablite so outleeri

In [47]:
df_num_rs = df_num.copy()

In [48]:

robscal = RobustScaler()
columnsO = ['Months_on_book','Credit_Limit', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Ct_Chng_Q4_Q1']

df_num_rs[columnsO] = robscal.fit_transform(df_num_rs[columnsO])

In [49]:
from sklearn.preprocessing import StandardScaler # za varijablite bez outleeri
standscal = StandardScaler()

In [50]:
columnsNO = ['Customer_Age', 'Dependent_count', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
            'Total_Revolving_Bal', 'Total_Trans_Ct', 'Avg_Utilization_Ratio']

df_num_rs[columnsNO] = standscal.fit_transform(df_num_rs[columnsNO])

In [51]:
target = df.Attrition_Flag
target = target.replace({'Existing Customer': '0', 'Attrited Customer': '1'})
y1 = target.copy()
df_cat_cb = df_cat.copy()

In [52]:
#skaliranje so CatBoost
import category_encoders as ce

cat_columns = ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category','Card_Category']

catBoostEncoder = ce.CatBoostEncoder(cols=cat_columns)

df_cat_cb = catBoostEncoder.fit_transform(df_cat_cb, y1)

#### Here we merge numerical and categorical encoded datasets

In [53]:
frames = [df_num_rs, df_cat_cb]

X_cb = pd.concat(frames, axis = 1)

In [54]:
preprocessed_sample = X_cb.drop(['CLIENTNUM', 'Attrition_Flag', 'Avg_Open_To_Buy','Credit_Limit','Customer_Age','Dependent_count','Months_on_book'], axis = 1)
# X1_new_train

In [56]:
!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier

## 2
#best_model = picke.load(.)

In [58]:
import pickle

In [59]:
filename = 'XGB_best_model'
XGB_best_model = pickle.load(open(filename, 'rb'))

# 3
XGB_best_model.predict(preprocessed_sample)

In [61]:
XGB_best_model.predict(preprocessed_sample)

array(['0'], dtype=object)